# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [4]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [5]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('../Vacation_Search/WeatherPy_vacation.csv')

# Display sample data
vacation_df.head(10)

,City,Country,Max Temp,Description,Lat,Lon,Hotel Name
0,lorengau,PG,80.44,moderate rain,-2.0226,147.2712,Seeadler Bay Hotel
1,hermanus,ZA,80.80,clear sky,-34.4187,19.2345,Aloe guest house
2,rikitea,PF,76.32,overcast clouds,-23.1203,-134.9692,Chez Bianca & Benoit
3,humaita,BR,72.27,overcast clouds,-7.5061,-63.0208,Humaita Quality Hotel
4,kapaa,US,71.58,clear sky,22.0752,-159.3190,Pono Kai Resort
5,cabo san lucas,MX,69.51,overcast clouds,22.8909,-109.9124,Comfort Rooms
6,puerto ayora,EC,71.55,scattered clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
7,puerto leguizamo,CO,70.12,overcast clouds,-0.1934,-74.7819,Hotel Girasoles
8,mana,GF,76.48,overcast clouds,5.6592,-53.7767,Le Samana
9,bredasdorp,ZA,77.74,clear sky,-34.5322,20.0403,Victoria Hotel


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [7]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    'Lon',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Max Temp',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country', 'Description']
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Max Temp,Hotel Name,Country,Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [42]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City'] == 'tingo maria']
vacation_end = vacation_df.loc[vacation_df['City'] == 'tingo maria']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'huarmey']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'hualmay']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'pisco']

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [43]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index = True)

# Display sample data
itinerary_df

,City,Country,Max Temp,Description,Lat,Lon,Hotel Name
0,tingo maria,PE,67.96,overcast clouds,-9.2895,-76.0088,La Gran Muralla II
1,huarmey,PE,69.71,overcast clouds,-10.0681,-78.1522,Hostal Santa Rosa
2,hualmay,PE,69.06,overcast clouds,-11.0964,-77.6139,Hotel Mi Choacan
3,pisco,PE,68.05,smoke,-13.7000,-76.2167,La Portada
4,tingo maria,PE,67.96,overcast clouds,-9.2895,-76.0088,La Gran Muralla II


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [44]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['City','Lon', 'Lat']].copy()

# Display sample data
waypoints_df

,City,Lon,Lat
0,tingo maria,-76.0088,-9.2895
1,huarmey,-78.1522,-10.0681
2,hualmay,-77.6139,-11.0964
3,pisco,-76.2167,-13.7000
4,tingo maria,-76.0088,-9.2895


## Use GeoViews to create map that shows the four cities in the itinerary

In [72]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    'Lon',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 100,
    scale = 1,
    color = 'City',
)

In [73]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [82]:
# Set parameters to trace the route
radius = 10000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [83]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lon"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '213fa99a52824a5599c9c7b756de843d',
 'waypoints': '-9.2895,-76.0088|-10.0681,-78.1522|-11.0964,-77.6139|-13.7,-76.2167|-9.2895,-76.0088'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [84]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-76.0088, -9.2895], 'original_index': 0},
     {'location': [-78.1522, -10.0681], 'original_index': 1},
     {'location': [-77.6139, -11.0964], 'original_index': 2},
     {'location': [-76.2167, -13.7], 'original_index': 3},
     {'location': [-76.0088, -9.2895], 'original_index': 4}],
    'units': 'metric',
    'distance': 1815958,
    'distance_units': 'meters',
    'time': 91251.333,
    'legs': [{'distance': 575966,
      'time': 29490.988,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 18,
        'time': 1.901,
        'instruction': {'text': 'Drive east on Jirón Jorge Basadre.'}},
       {'from_index': 1,
        'to_index': 32,
        'distance': 1283,
        'time': 93.634,
        'instruction': {'text': 'Turn right onto Avenida José Carlos Mariátegui.'}},
       {'from_index': 32,
        'to_index': 48,
        'distance': 452,
        'time':

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [102]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']

for coord in legs[3]:
    print(coord)

[-76.216619, -13.700003]
[-76.216621, -13.700055]
[-76.216705, -13.701513]
[-76.216709, -13.701653]
[-76.216792, -13.702891]
[-76.216918, -13.704306]
[-76.217011, -13.704916]
[-76.217229, -13.705772]
[-76.217456, -13.706663]
[-76.217492, -13.706805]
[-76.217617, -13.707294]
[-76.217621, -13.707458]
[-76.217855, -13.708531]
[-76.21805, -13.709371]
[-76.217056, -13.709437]
[-76.217016, -13.70947]
[-76.216464, -13.709507]
[-76.215381, -13.709541]
[-76.214785, -13.70956]
[-76.213863, -13.709589]
[-76.21341, -13.709603]
[-76.213367, -13.709604]
[-76.212569, -13.709629]
[-76.211537, -13.709662]
[-76.211482, -13.709663]
[-76.210683, -13.709689]
[-76.209943, -13.709712]
[-76.209909, -13.709716]
[-76.209883, -13.709727]
[-76.20986, -13.709746]
[-76.209834, -13.709777]
[-76.209831, -13.709786]
[-76.209802, -13.709836]
[-76.20976, -13.709877]
[-76.209708, -13.709904]
[-76.209649, -13.709915]
[-76.209593, -13.70991]
[-76.20954, -13.70989]
[-76.209494, -13.709856]
[-76.20946, -13.709812]
[-76.20941

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [117]:
# Create and empty list to store the longitude of each step
latitude = []

# Create and empty list to store the latitude of step
longitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coord in leg:
        latitude.append(coord[1])
        longitude.append(coord[0])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [118]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df['longitude'] = longitude
route_df['latitude'] = latitude

# Display sample data
route_df

,longitude,latitude
0,-76.008799,-9.289399
1,-76.008631,-9.289400
2,-76.008619,-9.289482
3,-76.008591,-9.289567
4,-76.008326,-9.290225
...,...,...
29580,-76.008326,-9.290225
29581,-76.008591,-9.289567
29582,-76.008619,-9.289482
29583,-76.008631,-9.289400


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [119]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path([route_df]).opts(line_width=2, color='red')
route_path

:Path   [longitude,latitude]

In [120]:
# Display a composed plot by using the route_map and route_path objects

waypoints_map*route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City)
   .Path.I   :Path   [longitude,latitude]